# Домашнее задание 2. Word2vec

## Работу выполнили Ковалев Евгений и Сухарев Иван, 4 группа

В данном домашнем задании мы работали с моделью word2vec: обучили ее на отзывах из файла unlabeledTrainData.tsv из соревнования "Bag of Words Meets Bags of Popcorn" на Kaggle (https://www.kaggle.com/c/word2vec-nlp-tutorial/data) и опробовали некоторые связанные с ней функции (.most_similar, .doesnt_match) для решения типовых задач (определение близких слов, ассоциаций, лишнего слова).

*[1 балл] Обучите модель word2vec. Оцените время обучения модели, используя модуль time.*

In [1]:
import pandas as pd
import time
from gensim.models import Word2Vec
import re
import nltk
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

В вышеупомянутом соревновании на Kaggle присутствует Overview (https://www.kaggle.com/c/word2vec-nlp-tutorial), где есть полезные туториалы, которые мы изучили. Для предобработки данных мы использовали описанный в одном из них класс KaggleWord2VecUtility (https://github.com/wendykan/DeepLearningMovies/blob/master/KaggleWord2VecUtility.py).

In [2]:
class KaggleWord2VecUtility(object):
    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        review_text = BeautifulSoup(review, "lxml").get_text()
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        words = review_text.lower().split()
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        return(words)

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        raw_sentences = tokenizer.tokenize(review.strip())
        sentences = []
        for raw_sentence in raw_sentences:
            if len(raw_sentence) > 0:
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        return sentences

# nltk.download('punkt')

Теперь выполним предобработку данных, используя токенизатор punkt.

In [3]:
data = pd.read_csv('unlabeledTrainData.tsv', sep='\t', error_bad_lines=False)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = []
for review in data["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

b'Skipping line 43043: expected 2 fields, saw 3\n'
/home/osboxes/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/home/osboxes/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/osboxes/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  '

Обучим модель.

In [4]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

print("Training Word2Vec model...")
start_time = time.time()
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, seed=1)
print('Time spent: {} minutes'.format((time.time() - start_time) / 60))
model.init_sims(replace=True)

Training Word2Vec model...
Time spent: 1.2908729275067647 minutes


Модель обучена!

*[2 балла] Приведите 5-10 примеров использования .most_similar для определения близких слов. Корректно ли они найдены? Являются ли синонимами исходного слова?*

Посмотрим, как модель справляется с определением близких слов к тем, что указаны ниже.

In [15]:
msWords = 'man, bird, awesome, cry, letter, leave, master, murder, poor, film'.split(', ')
for wd in msWords:
    closest = model.most_similar(positive = [wd], topn = 3)
    print(wd + ' : ' + ', '.join([c[0] for c in closest]))

man : woman, lad, lady
bird : motorcycle, goat, sheep
awesome : amazing, incredible, fantastic
cry : laugh, cringe, weep
letter : journal, letters, diary
leave : stay, reach, accept
master : puppet, masters, genius
murder : crime, murderer, murders
poor : lousy, terrible, weak
film : movie, picture, flick


Как видно, большинство слов найдено корректно. Однако в некоторых случаях вместо синонимов были найдены антонимы (man-woman, cry-laugh). Забавно, что самое близкое слово к "bird" - "motorcycle" (мы предположили, что они одинаково надоедливо шумят утром под окном, так что похоже на правду), а к "master" - "puppet" (возможно, это связано с известной песней группы "Metallica" - "Master of Puppets").

*[2 балла] Приведите 5-10 примеров использования .most_similar для определения ассоциаций (А к Б, как В к?). Корректно ли найдены ассоциации?*

Посмотрим, как модель справляется с определением ассоциаций, указанным ниже.

In [68]:
def closeAs(first, tosec, asIt):
    cl = model.most_similar([asIt, tosec], [first], topn=3)
    return first + ' -> ' + tosec + ', as ' + asIt + ' -> ' + cl[0][0]

assoClosest = [['day', 'light', 'night'], ['apple', 'fruit', 'car'], ['black', 'road', 'white'],
     ['moscow', 'russia', 'paris'], ['see', 'eyes', 'hear']]

for triple in assoClosest:
    print(closeAs(triple[0], triple[1], triple[2]))

day -> light, as night -> dark
apple -> fruit, as car -> traffic
black -> road, as white -> desert
moscow -> russia, as paris -> france
see -> eyes, as hear -> ears


Три (1, 4, 5) очевидные ассоциации найдены, несомненно, корректно, другие две посложнее, но тоже, на наш взгляд, вполне интерпретируемы (яблоко среди фруктов как машина в пробке, черная дорога как белая пустыня, почему бы и нет?)

*[2 балла] Приведите 5-10 примеров использования .doesnt_match для определения лишнего слова. Корректно ли найдены лишние слова?*

Посмотрим, как модель справляется с определением лишних слов в наборах, указанных ниже.

In [85]:
dmatch = 'potato apple carrot cucumber tomato\n\
earth mars mercury venus sun\n\
moscow paris london warsaw australia berlin\n\
horrible awesome flawless fabulous\n\
owl sparrow penguin eagle crow'.split('\n')

for line in dmatch:
    cl = line.split(' ')
    ans = model.doesnt_match(cl)
    print(line + ' -> ' + ans)

potato apple carrot cucumber tomato -> apple
earth mars mercury venus sun -> sun
moscow paris london warsaw australia berlin -> australia
horrible awesome flawless fabulous -> horrible
owl sparrow penguin eagle crow -> penguin


Все корректно. В первом наборе модель нашла фрукт среди овощей, во втором - звезду среди планет, в третьем - страну среди городов, в четвертом - негативную оценку среди позитивных, а в пятом даже определила птицу, которая не умеет летать.

*[3 балла] Попробуйте найти такие пары и тройки слов, для которых не выполняются/выполняются свойства коммутативности и транзитивности относительно операции определения близких слов.*

Коммутативные пары слов: guitar-piano, phone-cell

In [102]:
model.most_similar(['guitar'], topn = 3)

[('piano', 0.801474928855896),
 ('orchestra', 0.6634154319763184),
 ('drums', 0.6366580724716187)]

In [103]:
model.most_similar(['piano'], topn = 3)

[('guitar', 0.801474928855896),
 ('mozart', 0.633202075958252),
 ('orchestra', 0.6214369535446167)]

In [105]:
model.most_similar(['phone'], topn = 3)

[('cell', 0.6663293242454529),
 ('phones', 0.6594644784927368),
 ('telephone', 0.6335700154304504)]

In [106]:
model.most_similar(['cell'], topn = 3)

[('phones', 0.7089791893959045),
 ('mobile', 0.6965519189834595),
 ('phone', 0.6663293242454529)]

Некоммутативные пары слов: car-train, water-sand

In [93]:
model.most_similar(['car'], topn = 3)

[('truck', 0.77348792552948),
 ('train', 0.7148220539093018),
 ('bus', 0.6981351375579834)]

In [92]:
model.most_similar(['train'], topn = 3)

[('bus', 0.7943506836891174),
 ('helicopter', 0.7562844753265381),
 ('boat', 0.7497224807739258)]

In [100]:
model.most_similar(['water'], topn = 3)

[('sand', 0.7181669473648071),
 ('ocean', 0.7157705426216125),
 ('fish', 0.7005954384803772)]

In [101]:
model.most_similar(['sand'], topn = 3)

[('mud', 0.7529021501541138),
 ('trucks', 0.743501603603363),
 ('ocean', 0.7283374667167664)]

Транзитивные тройки слов: flick-movie-film, awesome-incredible-amazing

In [112]:
model.most_similar(['film'], topn = 3)

[('movie', 0.839878261089325),
 ('picture', 0.6457217931747437),
 ('flick', 0.640449583530426)]

In [113]:
model.most_similar(['movie'], topn = 3)

[('film', 0.8398780822753906),
 ('flick', 0.7002795934677124),
 ('movies', 0.565382182598114)]

In [114]:
model.most_similar(['amazing'], topn = 3)

[('incredible', 0.7846701145172119),
 ('awesome', 0.7521750926971436),
 ('outstanding', 0.6853843331336975)]

In [115]:
model.most_similar(['incredible'], topn = 3)

[('amazing', 0.7846700549125671),
 ('exceptional', 0.7585932612419128),
 ('awesome', 0.7159633636474609)]

Нетранзитивные тройки слов: fried-sugar-chocolate, fur-yellow-red

In [116]:
model.most_similar(['chocolate'], topn = 3)

[('soda', 0.6621962189674377),
 ('sugar', 0.6265168786048889),
 ('blanket', 0.6231487393379211)]

In [117]:
model.most_similar(['sugar'], topn = 3)

[('coated', 0.6561993956565857),
 ('chocolate', 0.6265168190002441),
 ('fried', 0.6079890727996826)]

In [121]:
model.most_similar(['red'], topn = 3)

[('yellow', 0.7565332651138306),
 ('blue', 0.732541561126709),
 ('velvet', 0.6416245102882385)]

In [122]:
model.most_similar(['yellow'], topn = 3)

[('fur', 0.7782459259033203),
 ('helmet', 0.7574673891067505),
 ('red', 0.7565332651138306)]